## Create file for orbital mechanics data related to Sun, Earth, and Moon. Outputs Parquet. 1996-jan-1 -> 2025-dec-31

In [ ]:
import numpy as np
import pandas as pd
from skyfield.api import load, wgs84

# --- Ephemeris and times ---
ts = load.timescale()
eph = load('de421.bsp')

dt_range = pd.date_range('1996-01-01', '2025-12-31 23:00', freq='h')
n_hours = len(dt_range)
times = ts.utc(
    dt_range.year.values.astype(int),
    dt_range.month.values.astype(int),
    dt_range.day.values.astype(int),
    dt_range.hour.values.astype(int),
)

G = 6.67430e-11
MASS_SUN_KG = 1.98847e30
MASS_EARTH_KG = 5.9722e24
MASS_MOON_KG = 7.34767309e22
EARTH_RADIUS_KM = 6371.0
EARTH_RADIUS_M = EARTH_RADIUS_KM * 1000

columns = [
    'sun_x_AU', 'sun_y_AU', 'sun_z_AU',
    'earth_x_AU', 'earth_y_AU', 'earth_z_AU',
    'moon_x_AU', 'moon_y_AU', 'moon_z_AU',
    'moon_hx_AU', 'moon_hy_AU', 'moon_hz_AU',
    'sun_vec_x', 'sun_vec_y', 'sun_vec_z',
    'moon_vec_x', 'moon_vec_y', 'moon_vec_z',
    'subsolar_lat', 'subsolar_lon',
    'sublunar_lat', 'sublunar_lon',
    'earth_obliquity_deg_true',
    'solar_tidal_potential_subsolar',
    'lunar_tidal_potential_sublunar',
    'mass_sun_kg', 'mass_earth_kg', 'mass_moon_kg'
]
arr = np.zeros((n_hours, len(columns)), dtype=np.float64)

earth = eph['earth']
moon = eph['moon']
sun = eph['sun']

earth_pos = earth.at(times).observe(sun).apparent().position.au
earth_helio = -earth_pos
moon_geo = earth.at(times).observe(moon).apparent().position.au
moon_helio = earth_helio + moon_geo

arr[:, 0:3] = 0.0
arr[:, 3:6] = earth_helio.T
arr[:, 6:9] = moon_geo.T
arr[:, 9:12] = moon_helio.T

sun_vec = earth_pos / np.linalg.norm(earth_pos, axis=0)
moon_vec = moon_geo / np.linalg.norm(moon_geo, axis=0)
arr[:, 12:15] = sun_vec.T
arr[:, 15:18] = moon_vec.T

subsolar = wgs84.subpoint(earth.at(times).observe(sun))
arr[:, 18] = subsolar.latitude.degrees
arr[:, 19] = subsolar.longitude.degrees

sublunar = wgs84.subpoint(earth.at(times).observe(moon))
arr[:, 20] = sublunar.latitude.degrees
arr[:, 21] = sublunar.longitude.degrees

# --- Mean obliquity using IAU 2000B (arcsec) then degrees (robust for all Skyfield) ---
jd = times.tt
T = (jd - 2451545.0)/36525.0
eps_arcsec = 84381.406 \
    - 46.836769 * T \
    - 0.0001831 * T**2 \
    + 0.00200340 * T**3 \
    - 0.000000576 * T**4 \
    - 0.0000000434 * T**5
arr[:, 22] = eps_arcsec / 3600.0

sun_pos_geo = earth.at(times).observe(sun).apparent().distance().km
moon_pos_geo = earth.at(times).observe(moon).apparent().distance().km
solar_tide = G * MASS_SUN_KG / (sun_pos_geo*1e3)**3 * EARTH_RADIUS_M**2
lunar_tide = G * MASS_MOON_KG / (moon_pos_geo*1e3)**3 * EARTH_RADIUS_M**2
arr[:, 23] = solar_tide
arr[:, 24] = lunar_tide

arr[:, 25] = MASS_SUN_KG
arr[:, 26] = MASS_EARTH_KG
arr[:, 27] = MASS_MOON_KG

df_out = pd.DataFrame(arr, columns=columns, index=dt_range)
df_out.to_parquet('sun_earth_moon_physics_ai.parquet')
print(df_out.head())
print(df_out.columns)
